# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [22]:
# webdriver -> Chrome -> chromedriver -> colar na pasta onde instalou o python
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# navegador = webdriver.Chrome("chromedriver.exe") -> Quando o chromedriver tiver no mesmo codigo em python
navegador = webdriver.Chrome() 


# Passo 1: Pegar a cotação do dolar


# copiando o codigo da busca do google
navegador.get("https://www.google.com.br/")

navegador.find_element(By.XPATH, 
                        '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar") 

navegador.find_element(By.XPATH, 
                        '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,
                      '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)
# Passo 2: Pegar a cotação do euro
navegador.get("https://www.google.com.br/")

navegador.find_element(By.XPATH, 
                        '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro") 

navegador.find_element(By.XPATH, 
                        '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
                      '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

# Passo 3: Pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.') # Trocar a virgula por ponto

print(cotacao_ouro)

navegador.quit()

5.5671
6.305269624
324.57


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [23]:
# Passo 4: Importar a base de dados e atualizar as cotações da minha base
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [25]:
# Passo 5: Calcular os novos preços e salvar/exportar a base de dados
### tabela.loc[linha, coluna]

tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)


# Atualizar as colunas

# Preço de compra = Preço Original * Cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# Preço de venda = Preço de Compra * Margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.56710,5567.044329,1.40,7793.862061
1,Carro Renault,4500.00,Euro,6.30527,28373.713308,2.00,56747.426616
2,Notebook Dell,899.99,Dólar,5.56710,5010.334329,1.70,8517.568359
3,IPhone,799.00,Dólar,5.56710,4448.112900,1.70,7561.791930
4,Carro Fiat,3000.00,Euro,6.30527,18915.808872,1.90,35940.036857
5,Celular Xiaomi,480.48,Dólar,5.56710,2674.880208,2.00,5349.760416
6,Joia 20g,20.00,Ouro,324.57000,6491.400000,1.15,7465.110000


### Agora vamos exportar a nova base de preços atualizada

In [27]:
tabela.to_excel("Produtos_Tabela_Tratada.xlsx", index=False)